In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [3]:
# Load data
# df = pd.read_csv("alevel_dataset.csv")
df = pd.read_csv("simulated_alevel_dataset_5.csv")

In [4]:
df.head(10)

,student_id,academic_year,gender,school_location,residence_location,school_type,is_boarding,revision_hours_per_day,distance_to_school_km,has_electricity,...,S6_Geography,S4_Economics,S5_Economics,S6_Economics,S4_Computer,S5_Computer,S6_Computer,S4_History,S5_History,S6_History
0,S00001,1994,female,urban,rural,public,no,2.34,0.82,yes,...,61.0,60.6,55.6,59.2,52.4,51.6,53.5,70.0,72.7,72.1
1,S00002,1994,male,urban,urban,public,yes,4.05,0.00,yes,...,62.6,66.7,70.7,74.2,69.5,71.5,70.4,66.5,62.6,62.9
2,S00003,1994,female,rural,rural,private,no,1.46,1.59,no,...,71.9,73.4,77.3,77.3,77.1,80.6,83.0,71.1,72.2,76.3
3,S00004,1994,female,rural,rural,public,no,2.03,1.84,no,...,68.8,66.8,68.9,76.4,65.6,71.9,72.5,64.2,65.7,67.6
4,S00005,1994,male,urban,rural,public,no,1.99,1.81,no,...,62.5,64.3,61.9,64.4,61.6,64.1,67.9,68.4,66.8,69.3
5,S00006,1994,female,urban,urban,private,no,2.65,1.45,yes,...,66.0,65.5,68.3,74.3,60.5,58.8,59.3,64.9,68.1,67.2
6,S00007,1994,male,urban,urban,public,yes,4.02,0.00,yes,...,61.1,65.5,68.7,74.4,61.0,64.9,66.6,70.5,70.5,69.5
7,S00008,1994,female,rural,rural,public,yes,3.76,0.00,no,...,69.1,66.0,69.1,66.0,66.2,65.0,64.9,82.7,83.0,86.9
8,S00009,1994,male,urban,rural,private,no,1.38,0.78,no,...,62.3,66.6,66.7,72.6,69.8,71.7,70.9,61.2,57.7,55.5
9,S00010,1994,male,rural,rural,private,yes,2.57,0.00,yes,...,60.3,65.0,62.0,62.1,69.1,73.4,74.8,63.9,64.9,67.7


In [5]:
# Encode categorical columns
cat_cols = ["gender", "school_location", "residence_location", "school_type",
            "is_boarding", "has_electricity", "parent_status", "extracurricular"]

for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

In [6]:
# Target encoding
target_col = "predicted_university_subject"
target_encoder = LabelEncoder()
df[target_col] = target_encoder.fit_transform(df[target_col])

In [7]:
# Save for future decoding
joblib.dump(target_encoder, "target_encoder.pkl")

['target_encoder.pkl']

In [8]:
# Split features and target
X = df.drop(columns=["student_id", "predicted_university_subject"])
y = df["predicted_university_subject"]

In [9]:
# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2, random_state=42)

#  LSTM Model for Sequential Learning

In [11]:
# Prepare time series data
def reshape_for_lstm(df, subjects):
    sequences = []
    for i in range(len(df)):
        seq = []
        for year in ["S4", "S5", "S6"]:
            seq.append([df[f"{year}_{sub}"].iloc[i] for sub in subjects])
        sequences.append(seq)
    return np.array(sequences)


subjects = ["Math", "Physics", "Chemistry", "Biology", "Geography", "Economics", "Computer", "History"]
X_seq = reshape_for_lstm(df, subjects)
y_seq = y.values

X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(X_seq, y_seq, stratify=y_seq, test_size=0.2)


In [12]:
# Build LSTM model
lstm_model = models.Sequential([
    layers.Input(shape=(3, len(subjects))),
    layers.LSTM(64, return_sequences=False),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(y_seq)), activation='softmax')
])

In [13]:
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_seq_train, y_seq_train, validation_split=0.2, epochs=30, batch_size=32)


Epoch 1/30


2400/2400 [==============================] - 19s 7ms/step - loss: 0.9855 - accuracy: 0.5992 - val_loss: 0.7447 - val_accuracy: 0.7122
Epoch 2/30
2400/2400 [==============================] - 12s 5ms/step - loss: 0.5582 - accuracy: 0.7770 - val_loss: 0.4276 - val_accuracy: 0.8320
Epoch 3/30
2400/2400 [==============================] - 12s 5ms/step - loss: 0.4433 - accuracy: 0.8219 - val_loss: 0.4824 - val_accuracy: 0.8083
Epoch 4/30
2400/2400 [==============================] - 26s 11ms/step - loss: 0.3883 - accuracy: 0.8415 - val_loss: 0.3009 - val_accuracy: 0.8787
Epoch 5/30
2400/2400 [==============================] - 27s 11ms/step - loss: 0.3717 - accuracy: 0.8477 - val_loss: 0.3218 - val_accuracy: 0.8714
Epoch 6/30
2400/2400 [==============================] - 32s 13ms/step - loss: 0.3419 - accuracy: 0.8597 - val_loss: 0.4003 - val_accuracy: 0.8382
Epoch 7/30
2400/2400 [==============================] - 25s 10ms/step - loss: 0.3369 - accuracy: 0.8602 - val_loss: 0.2525 -

# Random Forest (Baseline Tabular Model)

In [14]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      2822
           1       0.96      0.98      0.97      4135
           2       0.96      0.98      0.97      2684
           3       0.96      0.99      0.98      8425
           4       0.92      0.89      0.90      3456
           5       0.93      0.83      0.88      2478

    accuracy                           0.95     24000
   macro avg       0.94      0.93      0.94     24000
weighted avg       0.95      0.95      0.95     24000



#  ARIMA (Optional for Subject-Specific Score Forecasting)

In [15]:
# Example: forecasting a single student's Math score sequence
student_seq = df.iloc[0][["S4_Physics", "S5_Physics", "S6_Physics"]].astype(float).values
model = sm.tsa.ARIMA(student_seq, order=(1, 1, 0))
model_fit = model.fit()
forecast = model_fit.forecast()
print("Forecasted S7_Math:", forecast[0])

Forecasted S7_Math: 66.89762893346824


# Ensemble Meta-Model

In [16]:
# Example: combine LSTM and RF predictions
lstm_preds = lstm_model.predict(X_seq_test)
lstm_preds_cls = np.argmax(lstm_preds, axis=1)

# Stack and train a meta model
meta_X = np.vstack([lstm_preds_cls, rf.predict(X_test)]).T
meta_model = LogisticRegression()
meta_model.fit(meta_X, y_test)

meta_preds = meta_model.predict(meta_X)
print("Ensemble Accuracy:", accuracy_score(y_test, meta_preds))

750/750 [==============================] - 2s 2ms/step
Ensemble Accuracy: 0.9497916666666667


C:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

# Predict with meta model
meta_preds = meta_model.predict(meta_X)

# Accuracy
print("=== Ensemble Model Evaluation ===")
print("Accuracy:", accuracy_score(y_test, meta_preds))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, meta_preds))

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, meta_preds))

# Optional: ROC AUC for binary classification
if len(np.unique(y_test)) == 2:
    meta_probs = meta_model.predict_proba(meta_X)[:, 1]  # probability for class 1
    print("ROC AUC:", roc_auc_score(y_test, meta_probs))

=== Ensemble Model Evaluation ===
Accuracy: 0.9497916666666667

Confusion Matrix:
[[2634    0    0  129   43   16]
 [   0 4063    0    0   57   15]
 [   0    0 2641    0   34    9]
 [  59    0    0 8330   14   22]
 [  75   88   67   60 3082   84]
 [  47   83   57  119  127 2045]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      2822
           1       0.96      0.98      0.97      4135
           2       0.96      0.98      0.97      2684
           3       0.96      0.99      0.98      8425
           4       0.92      0.89      0.90      3456
           5       0.93      0.83      0.88      2478

    accuracy                           0.95     24000
   macro avg       0.94      0.93      0.94     24000
weighted avg       0.95      0.95      0.95     24000



In [18]:
# Assume you've trained these already
joblib.dump(rf, "rf_model.pkl")
joblib.dump(meta_model, "meta_model.pkl")
joblib.dump(scaler, "scaler.pkl")
lstm_model.save("lstm_model.h5")
joblib.dump(target_col, "target_encoder.pkl")

C:\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['target_encoder.pkl']